In [222]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pylab as plt 
import plotly as py 
import plotly.graph_objs as go 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pandas.plotting import lag_plot, autocorrelation_plot
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from time import time
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import math
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
import holidays
import itertools
from tqdm import tqdm
import pandas as pd
import plotly.graph_objects as go

In [223]:
def sin_cos_transform(series, divisor=-1):
    """
    Apply sin cos transformation to get cyclical features.

    Args:
        series (iterable): Series to transform
        divisor (int): Divisor value. If it doesn't set, function gets max value of series.

    Returns:
        - sin_values (list) - Sin transformed values
        - cos_values (list) - Cos transformed values
    """
    if divisor == -1:
        divisor = series.max()
    sin_values = [math.sin((2 * math.pi * x) / divisor) for x in list(series)]
    cos_values = [math.cos((2 * math.pi * x) / divisor) for x in list(series)]
    return sin_values, cos_values

In [243]:
# Function to check if a date is a US holiday
def is_us_holiday(date):
    us_holidays = holidays.US(years=date.year)
    return date in us_holidays


def create_features(df):
    df = df.copy()
    df.index = pd.to_datetime(df.index)
    
    df["day_of_week_sin"], df["day_of_week_cos"] = sin_cos_transform(df.index.dayofweek, divisor=7)
    df["day_of_month_sin"], df["day_of_month_cos"] = sin_cos_transform(df.index.day, divisor=30)
    df["month_sin"], df["month_cos"] = sin_cos_transform(df.index.month, divisor=12)


    df['day_of_quarter'] = (df.index.dayofyear - 1) % 91 + 1
    df['week_of_month'] = df.index.day // 7 + 1
    df['quarter_of_year'] = df.index.quarter
    df['week_of_year'] = df.index.isocalendar().week.astype('int32')
    df["is_wknd"] = df.index.weekday // 5
    df['is_month_start'] = df.index.is_month_start.astype(int)
    df['is_month_end'] = df.index.is_month_end.astype(int)
    df['season'] = df['month'] % 12 // 3 + 1
    df['lag7_births'] = df['births'].shift(periods=7)
    df['lag14_births'] = df['births'].shift(periods=14)
    df['lag4_births'] = df['births'].shift(periods=4)
    df['lag6_births'] = df['births'].shift(periods=6)
    df['births_diff7'] = df['births'].shift(7) - df['births'].shift(14)
    window_size = 7
    df['rolling_std'] = df['births'].rolling(window=window_size, min_periods=1, closed= "left").std()

    # Calculate the rolling mean, max, and min for the relevant columns
    df['rolling_mean'] = df['births'].rolling(window=window_size, min_periods=1, closed= "left").mean()
    df['rolling_max'] = df['births'].rolling(window=window_size, min_periods=1, closed= "left").max()
    df['rolling_min'] = df['births'].rolling(window=window_size, min_periods=1, closed= "left").min()
    # Add 'is_us_holiday' column to DataFrame
    df['is_us_holiday'] = df.index.map(is_us_holiday).astype(int)
    return df


In [248]:
df = pd.read_csv("Births2015.csv")
df = df.drop(columns = ['Unnamed: 0', 'wday'], axis = 1)
df['date'] = pd.to_datetime(df['date'])
df.describe()
df = df.set_index('date')

In [250]:
df= create_features(df)

In [251]:
df = df.dropna()

In [252]:
FEATURES= ['day_of_week_sin', 'day_of_week_cos', 'day_of_month_sin',
       'day_of_month_cos', 'month_sin', 'month_cos', 'day_of_quarter', 'is_wknd',
       'lag7_births', 'lag14_births', 'rolling_std', 'rolling_mean',
       'rolling_max', 'rolling_min', 'is_us_holiday']
TARGET = 'births' 

In [253]:
train = df[:-60] #291
val = df[-60:-30] #30
test = df[-30:] #30

In [254]:
exog_train = train[FEATURES]    #EXOGENOUS INOUTS
endog_train = train[TARGET]
exog_val = val[FEATURES]
endog_val = val[TARGET]
exog_test = test[FEATURES]
endog_test = test[TARGET]

PARAMETER SEARCH

In [ ]:
p_values = range(0, 3)  # NUMBER OF AUTOREGRESSIVE LAGS 
q_values = range(0, 3)  # ORDER OF DIFFERENCING TERM
d_values = range(0, 3)  # NUMBER OF MOVING AVERAGE LAGS
P_values = range(0, 3)  # 
D_values = range(0, 3)  # 
Q_values = range(0, 3)  #

parameters = list(itertools.product(p_values, d_values, q_values, P_values, D_values, Q_values))
best_aic = float("inf")
best_params = None

for param in tqdm(parameters):
    try:
        # Fit the SARIMAX model with the current combination of parameters
        model = SARIMAX(endog_train, exog = exog_train, order=(param[0], param[1], param[2]),
                        seasonal_order=(param[3], param[4], param[5],7))
        results = model.fit()

        # Calculate the AIC score for the model
        aic = results.aic

        # Update the best AIC and parameters if the current model has a lower AIC
        if aic < best_aic:
            best_aic = aic
            best_params = param

    except Exception as e:
        continue



In [256]:
# Print the best parameters and AIC value
print("Best Parameters:", best_params)
print("AIC:", best_aic)

Best Parameters: (0, 0, 0, 0, 0, 1)
AIC: 4326.140892940939


In [257]:
my_order=(0,0,2)
my_seasonal_order=(0,1,2,7)
model = SARIMAX(endog_train, exog= exog_train, order= my_order, seasonal_order=my_seasonal_order)


c:\Users\ipek\Desktop\staj-DATABOSS\MLproject\.env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.

c:\Users\ipek\Desktop\staj-DATABOSS\MLproject\.env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.



In [258]:
model_fit = model.fit()

c:\Users\ipek\Desktop\staj-DATABOSS\MLproject\.env\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [259]:
exog_forecast = pd.concat([exog_val, exog_test])
len(exog_forecast)

60

In [261]:
forecast_val = model_fit.get_forecast(steps=len(exog_forecast), exog=exog_forecast)
forecast_preds= forecast_val.predicted_mean
val_preds = forecast_preds[:30]
test_preds = forecast_preds[30:]

In [262]:


# Assuming you have loaded your dataset into a DataFrame named 'df' and 'predictions' contains the predicted values

# Create the figure
fig = go.Figure()

# Plot the original data
fig.add_trace(go.Scatter(x=pd.DataFrame(endog_val).index, y=pd.DataFrame(endog_val)["births"], mode='lines', name='Original Data', line=dict(color='dodgerblue')))

# Plot the predicted data
fig.add_trace(go.Scatter(x=val_preds.index, y=val_preds, mode='lines', name='Predictions Validation', line=dict(color='orange')))

fig.add_trace(go.Scatter(x=pd.DataFrame(endog_train).index, y=pd.DataFrame(endog_train)["births"], mode='lines', name='Original Data', line=dict(color='cyan')))

# Plot the predicted data
#fig.add_trace(go.Scatter(x=train_preds.index, y=train_preds, mode='lines', name='Predictions Train', line=dict(color='black')))


# Add layout information
fig.update_layout(title='US Births in 2015',
                  xaxis_title='Date',
                  yaxis_title='Births',
                  showlegend=True,
                  legend=dict(x=0.8, y=1),
                  height=400,
                  width=1000,
              #  xaxis=dict(range=['2015-01-01', '2015-12-31'])
                )  # Limit x-axis to the year 2015


# Show the plot
fig.show()


In [263]:

#print(f"train_MSE: {mse(train_preds, y_train)}")
#print(f"train_MAE: {mae(train_preds, y_train)}")
print(f"val_MSE: {mse(val_preds, endog_val)}")
print(f"val_MAE: {mae(val_preds, endog_val)}")
# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(endog_val, val_preds))

print("Root Mean Squared Error (RMSE):", rmse)

val_MSE: 951871.0067302665
val_MAE: 502.8728990715701
Root Mean Squared Error (RMSE): 975.6387685666589


FORECASTING

In [264]:
fig = go.Figure()

# Plot the original data
fig.add_trace(go.Scatter(x=pd.DataFrame(endog_test).index, y=pd.DataFrame(endog_test)["births"], mode='lines', name='Original Data', line=dict(color='dodgerblue')))

# Plot the predicted data
fig.add_trace(go.Scatter(x=test_preds.index, y=test_preds, mode='lines', name='Forecasting', line=dict(color='orange')))

# Add layout information
fig.update_layout(title='Forecasting US Births',
                  xaxis_title='Date',
                  yaxis_title='Births',
                  showlegend=True,
                  legend=dict(x=0.8, y=1),
                  height=400,
                  width=1000,
              #  xaxis=dict(range=['2015-01-01', '2015-12-31'])
                )  # Limit x-axis to the year 2015


# Show the plot
fig.show()

In [278]:
naive = pd.concat([endog_val, endog_test])
naive = naive[-37:-7]

In [283]:
# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(endog_test, test_preds))

print("Root Mean Squared Error (RMSE) for test:", rmse)


rmse2 = np.sqrt(mean_squared_error(endog_test, naive))
print("Root Mean Squared Error (RMSE) naive:", rmse2)


Root Mean Squared Error (RMSE) for test: 1117.8363959272986
Root Mean Squared Error (RMSE) naive: 1798.9476646083956
